In [1]:
using Random, Distributions
using LinearAlgebra, Statistics
using Plots
using CSV, DataFrames

In [2]:
# Parameters to choose

#γ = 0.05:0.05:0.2
β = 0.8 # 4% interest rate

ρ = 0.866
σ = 0.059
σ_0 = sqrt(σ^2/(1 - ρ^2)) # 0.12
m = 0.088

m_0 = 0.9
f = 20
c_e = 15

α=0.2;

In [3]:
# Discretize AR(1) - Tauchen Method

function tauchen(m, σ, ρ=0.866, n_std = 7, N = 120)
    # Discretize space
    m_s = m/(1-ρ)
    σ_s = σ/sqrt(1-ρ^2)
    s_min = m_s - n_std * σ_s/2
    s_max = m_s + n_std * σ_s
    log_s = range(s_min, s_max, length = N)
    s = exp.(log_s)
    d = log_s[2] - log_s[1]
    d_half = d/2
    
    # Tranformation matrix
    P = zeros(N,N)
    for i in eachindex(log_s) 
        P[i,1] = cdf(Normal(0,1), (s_min + d_half -ρ*log_s[i]-m)/σ)
        P[i,N] = 1 - cdf(Normal(0,1), (s_max - d_half -ρ*log_s[i]-m)/σ)
        for j in 2:N-1
            P[i,j] = cdf(Normal(0,1), (log_s[j] + d_half -ρ*log_s[i]-m)/σ) - cdf(Normal(0,1), (log_s[j]  - d_half -ρ*log_s[i]-m)/σ)
        end    
    end
    return P,s
end

tauchen (generic function with 4 methods)

In [4]:
(P,s) = tauchen(m, σ);

In [5]:
# Find Value Function

function findV_g(p, V0, P, s, α=0.2, f=20, β=0.8, max_it = 1e4, tol=1e-8)
    i = 1 
    dist = 1e3
    V = V0
    N = length(s)
    
    # optimal q
    q = (p*s).^(1/α)
    # profit function
    profit = p*q - (q.^(1+α))./((1+α)*s)-f*ones(N)
    
    while (i < max_it) & (dist > tol)
        V1 =  max.(0, profit + β * P'*V)
        dist = norm(V1 - V)
        V = V1
        i += 1
    end
    
    return V
    
end

findV_g (generic function with 6 methods)

In [6]:
function fr_entry_g(p, P, s, g, c_e=15)
    V0 = zeros(N)
    V = findV_g(p, V0, P, s)
    return  β*g'*V - c_e
end

fr_entry_g (generic function with 2 methods)

In [7]:
function eqm_price_g(P, s, g,  max_it = 1e3, tol=1e-8)
    p_max = 100
    p_min = 0
    v_e = 1e3
    i = 0
    
    while (i < max_it) & (p_max - p_min > tol)
        p = (p_max + p_min)/2
        v_e = fr_entry_g(p, P, s, g)
        if v_e > 0
            p_max = p
        else
            p_min = p
        end
        i +=1
    end
    
    return p_min
end

eqm_price_g (generic function with 3 methods)

In [8]:
# Exit threshold

function exit_s_g(p, s, P, α=0.2, f=20)
    v = -1
    # optimal q
    q = (p*s).^(1/α)
    # profit function
    profit = p*q - (q.^(1+α))./((1+α)*s)-f*ones(N)
    
    V0 = zeros(N)
    V = profit + β*P'*findV_g(p, V0, P, s)
    
    return sum(V.<0)
end

exit_s_g (generic function with 3 methods)

In [9]:
# moments 

N = length(s)
    
    # Initial distribution
    G = cdf(LogNormal(m_0,σ_0),s)
    g = zeros(N)
    g[1] = G[1]

    for i in 2:N
        g[i] = G[i] - G[i-1]
    end
    g=round.(g/sum(g),digits=4);

In [10]:
price = eqm_price_g(P, s, g)   
q = (price*s).^(1/0.2)

# Exit rule
k_s = exit_s_g(price,s, P)

# cross section distribution
S0 = 1 - G[k_s]
μ0 = zeros(N)
μ0[k_s:N]=g[k_s:N]./S0
q0 = q'*μ0


Prob = zeros(N)
for i in 1:N
    Prob[i] = sum(P[i,k_s:N])
end

# Average output
S1 = S0 * Prob'*μ0 
μ1 = P'*μ0*S0/S1
q1 = q'*μ1


S2 = S1 * Prob'*μ1 
μ2 = P'*μ1*S1/S2
q2 = q'*μ2

S3 = S2 * Prob'*μ2 
μ3 = P'*μ2*S2/S3
q3 = q'*μ3

S4 = S3 * Prob'*μ3 
μ4 = P'*μ3*S3/S4
q4 = q'*μ4

S5 = S4 * Prob'*μ4 
μ5 = P'*μ4*S4/S5
q5 = q'*μ5

# average cost
cost = (q.^(1+α))./((1+α)*s)-f*ones(N);

ac0 = cost'*μ0/q0
ac1 = cost'*μ1/q1
ac2 = cost'*μ2/q2
ac3 = cost'*μ3/q3
ac4 = cost'*μ4/q4
ac5 = cost'*μ5/q5;

In [11]:
print(q0," ",q1," ",q2," ",q3," ",q4," ",q5)

69.61399830649643 71.82139238235601 76.86757103652178 84.13328184427829 93.53956880613573 104.91296373593721

In [12]:
print(S0," ",S1," ",S2," ",S3," ",S4," ",S5)

0.5712485435069523 0.45579549066680813 0.35902761046251297 0.28243814282715224 0.22285947250694202 0.17721303741098787

In [13]:
print(S0*q0," ",S1*q1," ",S2*q2," ",S3*q3," ",S4*q4," ",S5*q5)

39.76689514028153 32.73586678128931 27.597580351299886 23.762447874051325 20.84617896266222 18.591944967434255

In [14]:
print(price/ac0," ",price/ac1," ",price/ac2," ",price/ac3," ",price/ac4," ",price/ac5)

2.1004525582799562 2.50401586439219 3.1391418342851067 4.246708838082343 6.585901924208639 14.44881978015842

In [15]:
γ = 1:0.1:1.5

1.0:0.1:1.5

In [16]:
# Turnover - exit
xi = zeros(6)
for i in 1:6
    xi[i] = ((S0-S1)/γ[i]+(S1-S2)/γ[i]^2+(S2-S3)/γ[i]^3+(S3-S4)/γ[i]^4+(S4-S5)/γ[i]^5)/(S0 + S1/γ[i]+S2/γ[i]^2+S3/γ[i]^3+S4/γ[i]^4+S5/γ[i]^5)
end
print(xi)

[0.19048577694722563, 0.17731877934863624, 0.16518800621802887, 0.15417075366605723, 0.14424019435995325, 0.135317781044507]

In [17]:
# Turnover - entry
print(γ-ones(6)+xi)

[0.19048577694722563, 0.27731877934863636, 0.3651880062180288, 0.4541707536660573, 0.5442401943599532, 0.635317781044507]

In [18]:
df = DataFrame(CSV.File("/Users/yinyin/Documents/GitHub/Practice/firm_dynamics/bds2018_sector_fage.csv",delim=','))
d = filter([:year, :sector] => (year,sector) -> year==2018 && sector == "44-45" ,df)

,year,sector,fage,firms,estabs,emp,denom,estabs_entry
,Int64,String,String,Int64,Int64,Int64,Int64,String
1,2018,44-45,a) 0,43132,43449,193935,96968,43449
2,2018,44-45,b) 1,36043,36596,182127,185175,63
3,2018,44-45,c) 2,32769,33649,190511,192813,1189
4,2018,44-45,d) 3,28670,29817,188771,192942,1025
5,2018,44-45,e) 4,26064,27532,185633,187887,931
6,2018,44-45,f) 5,23174,24217,162477,165576,724
7,2018,44-45,g) 6 to 10,90776,97963,786885,802178,2477
8,2018,44-45,h) 11 to 15,74236,84918,763098,769537,1820
9,2018,44-45,i) 16 to 20,50111,63855,634190,644065,1500


In [19]:
firms = zeros(6) # 6 age cohorts: entrants, 1-5, 6-10, 11-15, 16-20, 21-25
emp = zeros(6)
deaths = zeros(6)

firms[1] = d.firms[1]*5
emp[1] = d.emp[1]*5
deaths[1] =0

firms[2] = sum(d.firms[2:6])
emp[2] = sum(d.emp[2:6])
deaths[2] = sum(parse.(Int64, d.firmdeath_firms[2:6]))

for i in 3:6
    firms[i] = d.firms[4+i]
    emp[i] = d.emp[4+i]
    deaths[i] = parse(Int64, d.firmdeath_firms[4+i])
end

avg_emp = emp./(firms-deaths); # average employment for surviving firms
lemp = log.(avg_emp);

In [20]:
m = zeros(6)
m[6] = firms[1]
m[5] = firms[2]/S1
m[4] = firms[3]/S2
m[3] = firms[4]/S3
m[2] = firms[5]/S4
m[1] = firms[6]/S5

lm = log.(m)
# log(m_{t+1}) = log(m_{t}) - log(γ)
exp(mean(lm[1:5])-mean(lm[2:6]))

1.018426315266197

In [21]:
γ=1.02
# Turnover
a = ((S0-S1)/γ+(S1-S2)/γ^2+(S2-S3)/γ^3+(S3-S4)/γ^4+(S4-S5)/γ^5)/(S0 + S1/γ+S2/γ^2+S3/γ^3+S4/γ^4+S5/γ^5)
print(γ-1+a)

0.20777854624312933

In [22]:
a

0.1877785462431293